In [1]:
import ee
import geemap

C:\Users\AMRIT\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ee.Authenticate()

Enter verification code: 4/1AfJohXmYWRWja36twbRnXtjyS7dNSBC221GiU8Jk7lziS-MqtaijxHRZwaM

Successfully saved authorization token.


In [3]:
import ee
ee.Initialize()
import numpy as np

In [4]:
rgb = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').select(['B4', 'B3', 'B2']).filterDate('2020-06-05', '2020-09-01').filterMetadata('CLOUD_COVER', 'less_than', 15).median().visualize(**{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5})
elevation = ee.Image('CGIAR/SRTM90_V4')

In [5]:
X = np.linspace(-120,-106,71)
Y = np.linspace(35,49,71)

In [6]:
scale_factors = np.zeros(25)

In [ ]:
i = 0
while i < 50:
    x = X[i]
    y = Y[i]

    region = ee.Geometry.Polygon([[[x, y+0.2], [x+0.2, y+0.2], [x, y], [x+0.2, y]]])

    rgb_task_config = {
        'scale': 30, 
        'region': region,
        'crs': 'EPSG:5070',
        'maxPixels': 1e13,
        'folder': 'RGB4'
    }

    elevation_task_config = {
        'scale': 30, 
        'region': region,
        'crs': 'EPSG:5070',
        'maxPixels': 1e13,
        'folder': 'Elevation4'
    }

    min_max = elevation.reduceRegion(
        reducer = ee.Reducer.minMax(),
        geometry = region,
        scale = 30,
        maxPixels = 1e13
    )

    min_i = min_max.getInfo()['elevation_min']
    max_i = min_max.getInfo()['elevation_max']

    rgb_task = ee.batch.Export.image(rgb, 'RGB_' + str(i), rgb_task_config)
    elevation_task = ee.batch.Export.image(
        elevation.visualize(**{'bands': ['elevation'], 'min': min_i, 'max': max_i}),
        'E_' + str(i),
        elevation_task_config
    )

    rgb_task.start()
    elevation_task.start()

    i += 1

np.save('scale_factors.npy', scale_factors)